## Logging on

Use Selenium to visit https://webapps1.chicago.gov/buildingrecords/ and accept the agreement.

> Think about when you use `.find_element...` and when you use `.find_elementSSS...`

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/angeline_jcq/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/9m/w7ffkstj051dnyjnyklwvwmr0000gn/T/ipykernel_3111/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
driver.get('https://webapps1.chicago.gov/buildingrecords/')

In [4]:
driver.find_element(By.ID, 'rbnAgreement1').click()

In [5]:
driver.find_element(By.ID, 'submit').click()

## Searching

Search for **400 E 41ST ST**.

In [6]:
driver.find_element(By.ID, 'fullAddress').send_keys('400 E 41ST ST')

In [7]:
driver.find_element(By.ID, 'submit').click()

## Saving tables with pandas

Use pandas to save a CSV of all **permits** to `Permits - 400 E 41ST ST.csv`. Note that there are **different sections of the page**, not just one long permits table.

> - *Tip: When using `.read_html`, try using `flavor='lxml'` and comparing the results to `flavor='html5lib'`. Which works better?*
> - *Tip: You might need to install `html5lib` using `pip`. If so, you'll need to restart the notebook using **Kernel > Restart** before it will work.*

In [8]:
permit = driver.find_element(By.ID, 'resultstable_permits')
#df = pd.read_html(permit.get_attribute('outerHTML'), flavor='lxml')[0]
df = pd.read_html(permit.get_attribute('outerHTML'), flavor='html5lib')[0]
df
#'html5lib' works better because it can return all the date issued

,PERMIT #,DATE ISSUED,DESCRIPTION OF WORK
0,100845718,10/11/2019,ERECT TWO SCAFFOLDS FROM 10/14/2019 TO 10/14/2...
1,100778302,08/17/2018,PERMIT EXPIRES ON 10/17/2018 Erection Starts: ...
2,100721255,08/24/2017,PERMIT EXPIRES ON 10/24/2017 ERECTION STARTS: ...
3,100693399,03/03/2017,INSTALLATION OF LOW VOLTAGE BURGLAR ALARM INTE...
4,100665436,08/24/2016,PERMIT EXPIRES ON 10/24/2016 ERECTION STARTS: ...
5,100610771,08/28/2015,PERMIT EXPIRES ON 10/28/2015 ERECTION STARTS: ...
6,100581991,02/18/2015,TRACE AND REPAIR BROKEN UNDERGROUND FEED TO EX...
7,100479194,04/16/2013,INTERNALLY LIT SIGN CABINET ON SOUTH ELEVATION
8,100385721,03/25/2011,RPACE CONCRETE SLAB WITH NEW AT GROUNGD FLOOR ...
9,100267298,12/04/2008,INTERIOR ALTERATIONS TO MEDICAL OFFICE SUITE 1...


## Saving tables the long way

Save a CSV of all DOB inspections to `Inspections - 400 E 41ST ST.csv`.

This is more complicated than the last one becuse **we also need to save the URL to the inspection** (see how the inspection number is a link?). As a result, you won't be able to use pandas! Instead, you'll need to use a loop and create a list of dictionaries.

You can use Selenium or you can feed the source to BeautifulSoup. You should have approximately 160 rows.

You'll probably need to find the table first, then the rows inside, then the cells inside of each row. You'll probably use lots of list indexing. I might recommend XPath for finding the table.

*Tip: If you get a "list index out of range" error, it's probably due to an issue involving `thead` vs `tbody` elements. What are they? What are they for? What's in them? There are a few ways to troubleshoot it.*

In [9]:
inspections = driver.find_element(By.ID, 'resultstable_inspections')

In [10]:
rows = inspections.find_elements(By.TAG_NAME, 'tr')
inspect_result = []
for row in rows[1:]:
    one_row = {}
    cells = row.find_elements(By.TAG_NAME, 'td')
    one_row['ID'] = cells[0].text
    one_row['Date'] = cells[1].text
    one_row['Status'] = cells[2].text
    one_row['Type_description'] = cells[3].text
    one_row['href'] = cells[0].find_element(By.TAG_NAME, 'a').get_attribute('href')
    inspect_result.append(one_row)
inspect_result

[{'ID': '13587202',
  'Date': '10/25/2021',
  'Status': 'FAILED',
  'Type_description': 'ANNUAL INSPECTION',
  'href': 'https://webapps1.chicago.gov/buildingrecords/inspectiondetails?addr=364923&insp=13587202'},
 {'ID': '12941650',
  'Date': '08/23/2021',
  'Status': 'FAILED',
  'Type_description': 'CONSERVATION ANNUAL',
  'href': 'https://webapps1.chicago.gov/buildingrecords/inspectiondetails?addr=364923&insp=12941650'},
 {'ID': '13528258',
  'Date': '08/09/2021',
  'Status': 'FAILED',
  'Type_description': 'ANNUAL INSPECTION',
  'href': 'https://webapps1.chicago.gov/buildingrecords/inspectiondetails?addr=364923&insp=13528258'},
 {'ID': '13346279',
  'Date': '03/03/2021',
  'Status': 'FAILED',
  'Type_description': 'ANNUAL INSPECTION',
  'href': 'https://webapps1.chicago.gov/buildingrecords/inspectiondetails?addr=364923&insp=13346279'},
 {'ID': '13175960',
  'Date': '11/30/2020',
  'Status': 'FAILED',
  'Type_description': 'ANNUAL INSPECTION',
  'href': 'https://webapps1.chicago.gov/b

In [11]:
df = pd.DataFrame(inspect_result)
df

,ID,Date,Status,Type_description,href
0,13587202,10/25/2021,FAILED,ANNUAL INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...
1,12941650,08/23/2021,FAILED,CONSERVATION ANNUAL,https://webapps1.chicago.gov/buildingrecords/i...
2,13528258,08/09/2021,FAILED,ANNUAL INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...
3,13346279,03/03/2021,FAILED,ANNUAL INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...
4,13175960,11/30/2020,FAILED,ANNUAL INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...
...,...,...,...,...,...
155,25836,07/09/2001,PASSED,SIGN ANNUAL INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...
156,130126,05/09/1997,CLOSED,FIRE PREVENTION PUMPS LEGACY,https://webapps1.chicago.gov/buildingrecords/i...
157,9475223,01/28/1997,CLOSED,ELEVATOR LEGACY INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...
158,9457580,01/21/1997,CLOSED,ELEVATOR LEGACY INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...


In [12]:
df.shape

(160, 5)

In [13]:
df.to_csv('Inspections - 400 E 41ST ST.csv', index=False)

### Bonus preview of Wednesday's content

**You prrrrrobably shouldn't do this one unless you want a real challenge.**

If you click the inspection number, it'll open up a new window that shows you details of the violations from that visit. Count the number of violations for each visit and save it in a new column called **num_violations**.

Save this file as `Inspections - 400 E 41ST ST - with counts.csv`.

Since clicking the link opens in a new window, we have to say "Hey Selenium, pay attention to that new window!" We do that with `driver.switch_to.window(driver.window_handles[-1])` (each window gets a `window_handle`, and we're just asking the driver to switch to the last one.). A rough sketch of what your code will look like is here:

```python
# Click the link that opens the new window

# Switch to the new window/tab
driver.switch_to.window(driver.window_handles[-1])

# Do your scraping in here

# Close the new window/tab
driver.close()

# Switch back to the original window/tab
driver.switch_to.window(driver.window_handles[0])
```

You'll want to play around with them individually before you try it with the whole set - the ones that pass are very different pages than the ones with violations! There are a few ways to get the number of violations, some easier than others.

In [14]:
#Method 1
inspections = driver.find_element(By.ID, 'resultstable_inspections')
rows = inspections.find_elements(By.TAG_NAME, 'tr')
list_violation = []
for row in rows[1:]:
        cells = row.find_elements(By.TAG_NAME, 'td')
        id_ = cells[0].text
        status = cells[2].text
        if status == 'FAILED':
            cells[0].click()
            driver.switch_to.window(driver.window_handles[-1])
            counts = driver.find_elements(By.TAG_NAME, 'tr')
            if len(counts) > 2:
                list_violation.append(len(counts) - 2)
            else:
                list_violation.append(0)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        else:
            list_violation.append(0)

In [15]:
df['num_violations'] = list_violation
df.head()

,ID,Date,Status,Type_description,href,num_violations
0,13587202,10/25/2021,FAILED,ANNUAL INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...,2
1,12941650,08/23/2021,FAILED,CONSERVATION ANNUAL,https://webapps1.chicago.gov/buildingrecords/i...,4
2,13528258,08/09/2021,FAILED,ANNUAL INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...,2
3,13346279,03/03/2021,FAILED,ANNUAL INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...,20
4,13175960,11/30/2020,FAILED,ANNUAL INSPECTION,https://webapps1.chicago.gov/buildingrecords/i...,10


In [16]:
df.to_csv("Inspections - 400 E 41ST ST - with counts.csv", index=False)

In [16]:
#Method 2
import re

inspections = driver.find_element(By.ID, 'resultstable_inspections')
rows = inspections.find_elements(By.TAG_NAME, 'tr')
for row in rows[1:]:
    row.find_element(By.TAG_NAME, 'a').click()
    driver.switch_to.window(driver.window_handles[-1])
    try:
        number_result = driver.find_element_by_xpath("/html/body/div/div[4]/div[4]/div/div[2]/div").text
        number = re.findall(r' (\d+) entries', number_result)
        print(number)
    except:
        print('NaN')
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

/var/folders/9m/w7ffkstj051dnyjnyklwvwmr0000gn/T/ipykernel_19517/337560004.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  number_result = driver.find_element_by_xpath("/html/body/div/div[4]/div[4]/div/div[2]/div").text


['2']
['4']
['2']
['20']
['10']
NaN
['6']
['3']
NaN
NaN
NaN
['1']
NaN
NaN
['1']
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
['4']
['6']
['3']
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
['1']
['4']
['1']
NaN
NaN
NaN
NaN
NaN
['1']
['1']
NaN
NaN
['6']
NaN
NaN
['1']
NaN
['4']
['1']
['1']
['1']
['2']
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
['4']
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
['1']
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
['2']
NaN
NaN
NaN
NaN
NaN
NaN
['1']
NaN
NaN
NaN
NaN
NaN
NaN


In [21]:
#Method 3
inspections = driver.find_element(By.ID, 'resultstable_inspections')
rows = inspections.find_elements(By.TAG_NAME, 'tr')
for row in rows[1:]:
    row.find_element(By.TAG_NAME, 'a').click()
    driver.switch_to.window(driver.window_handles[-1])
    try:
        counts = driver.find_elements(By.TAG_NAME, 'tr')
        if len(counts) == 0:
            print(0)
        else:
            print(len(counts)-2)
    except:
        print('NaN')
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

2
4
2
20
10
0
6
3
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
4
6
3
0
0
0
0
0
0
0
0
1
4
1
0
0
0
0
0
1
1
0
0
6
0
0
1
0
4
1
1
1
2
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
1
0
0
0
0
0
0
